## Imports

In [2]:
#Data Procssing
import pandas
import matplotlib.pylab as plt #numpy + pyplot as 1 library

#Image Processing
from PIL import Image
from matplotlib.pyplot import imshow

#File handling
import os
import glob

#Pytorch
import torch
from torch.utils.data import Dataset

#IBM library 
# import skillsnetworkb

ModuleNotFoundError: No module named 'torch'

## Dataset

**Link:** https://data.mendeley.com/datasets/5y9wdsg2zt/2

**Size:** 203MB

**Classes:** 2 - positive(cracked),negative(not cracked)

**Images:** 40000 , 227 x 227, RGB channels.

## Helper Functions

Function to plot images

In [2]:
def show_data(data_sample, shape = (28, 28)):
    plt.imshow(data_sample[0].numpy().reshape(shape), cmap='gray')
    plt.title('y = ' + data_sample[1])

Function to plot image from path

In [6]:
def show_image(path,title):
    image = Image.open(path)
    plt.imshow(image)
    plt.title(title)
    plt.show()

Function to return list of images full path

In [4]:
def file_maker(directory,positive_directory,negative_directory):
    positive_file_path = os.path.join(directory,positive_directory)
    positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
    positive_files.sort()

    negative_file_path = os.path.join(directory,negative_directory)
    negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
    negative_files.sort()
    
    return positive_files,negative_files

Dataloader (Function to build Dataset)

In [1]:
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True):
        directory="/resources/data"
        positive="Positive"
        negative="Negative"
        
        #define paths
        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        
        #make list of full image paths
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        
        #sort lists
        positive_files.sort()
        negative_files.sort()
        
        #total number of images
        number_of_samples = len(positive_files) + len(negative_files)
        
        #Combine both list. Even = +ve, Odd = -ve
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        
        # The transform is goint to be used on image
        self.transform = transform
        
        #List of output class 
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        #Even = 1/positive, Odd = 0/negative
        self.Y[::2]=1
        self.Y[1::2]=0
        
        
        #if train on [0-30000], val on [30000-40000]
        if train:
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:

            self.Y=self.Y[30000:]
            self.len=len(self.all_files)
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        #open image tensor and output
        image=Image.open(self.all_files[idx])
        y=self.Y[idx]
          
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)

        return image, y

NameError: name 'Dataset' is not defined